In [1]:

import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.layers import merge
# from keras.layers.merge import Concatenate
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization

from keras.optimizers import Adam, SGD



/Users/jx/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [3]:
import numpy as np
import h5py
import scipy.io
validmat = scipy.io.loadmat('/Users/jx/Documents/FACSseq/YFSIVc/sTRSV_valid.mat')
testmat = scipy.io.loadmat('/Users/jx/Documents/FACSseq/YFSIVc/sTRSV_valid.mat')
trainmat = scipy.io.loadmat('/Users/jx/Documents/FACSseq/YFSIVc/sTRSV_train.mat')


X_train = np.transpose(np.array(trainmat['tr'][0][0][0]),axes=(0,2,1))
# X_train = np.expand_dims(X_train,axis=(2))
y_train = np.array(trainmat['tr'][0][0][1]).squeeze()
# y_train = y_train.reshape((-1, 1))
# y_train.squeeze()
print X_train.shape
print y_train
print X_train


(61340, 4, 113)
[1 0 0 ..., 0 0 0]
[[[0 0 0 ..., 0 0 0]
  [0 0 1 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [1 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 1 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [1 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 1 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [1 0 0 ..., 0 0 0]]

 ..., 
 [[0 0 0 ..., 0 0 0]
  [0 0 1 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [1 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 1 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [1 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 1 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [1 0 0 ..., 0 0 0]]]


In [4]:


lr = 1e-5#learning rate
reg = 1e-3
print 'building model'
nb_filters=32
model = Sequential()
# model.add(LSTM(256,  W_regularizer=l2(reg),return_sequences=True, input_shape=(4,113)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# model.add(LSTM(40,  W_regularizer=l2(reg)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))


# model.add(LSTM(32,24, W_regularizer=l2(reg),input_shape=(X_train.shape[0], X_train.shape[1], X_train.shape[2]))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# conv1=Sequential()
model.add(Convolution1D(8,256, border_mode='same', W_regularizer=l2(reg),input_shape=(4,113))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Convolution1D(8,64, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Convolution1D(8,64, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Convolution1D(8,64, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.1))
# model.add(Convolution1D(8,64, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.1))

# model.add(Convolution1D(4,32, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.05))


model.add(Flatten())


# model.add(Dense(1000))
model.add(Dense(1))
model.add(Activation('sigmoid'))

adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

print 'compiling model'
# model.compile(loss='mse', optimizer='adam', metrics=["mse"])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])


checkpointer = ModelCheckpoint(filepath="bestmodel3D_classify.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

X_valid=np.transpose(validmat['tr'][0][0][0],axes=(0,2,1))
# X_valid=np.expand_dims(X_valid)
y_valid=np.array(validmat['tr'][0][0][1]).squeeze()


history=model.fit(X_train, y_train, batch_size=512, nb_epoch=500, shuffle=True, validation_data=(X_valid, y_valid),callbacks=[checkpointer,earlystopper])



building model
compiling model
Train on 61340 samples, validate on 6816 samples
Epoch 1/500
61340/61340 [==============================] - 209s - loss: 0.3334 - acc: 0.9124 - val_loss: 0.2830 - val_acc: 0.9136
Epoch 2/500
61340/61340 [==============================] - 209s - loss: 0.2807 - acc: 0.9124 - val_loss: 0.2677 - val_acc: 0.9136
Epoch 3/500
61340/61340 [==============================] - 208s - loss: 0.2713 - acc: 0.9123 - val_loss: 0.2633 - val_acc: 0.9136
Epoch 4/500
61340/61340 [==============================] - 212s - loss: 0.2671 - acc: 0.9123 - val_loss: 0.2602 - val_acc: 0.9136
Epoch 5/500
61340/61340 [==============================] - 210s - loss: 0.2640 - acc: 0.9124 - val_loss: 0.2582 - val_acc: 0.9136
Epoch 6/500
61340/61340 [==============================] - 212s - loss: 0.2621 - acc: 0.9126 - val_loss: 0.2562 - val_acc: 0.9139
Epoch 7/500
61340/61340 [==============================] - 212s - loss: 0.2604 - acc: 0.9121 - val_loss: 0.2572 - val_acc: 0.9140
Epoch 8/50

In [5]:
%matplotlib

out=model.predict(X_train, batch_size=512,verbose=1)
plt.plot(y_train,out,'ro')

model.summary()


Using matplotlib backend: MacOSX
61340/61340 [==============================] - 114s   
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 4, 8)          231432      convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 4, 8)          0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 4, 8)          0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 4, 8)          4104        dropout_1[0][0]                  
___

In [62]:
import scipy
out=out.squeeze()
print out.shape
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(y_train, out)
print r_value
print slope
print intercept

(12317,)
0.674227100021
0.407035704157
-0.0509881643101


In [7]:
%matplotlib




X_test=np.transpose(testmat['tr'][0][0][0],axes=(0,2,1))
y_test=np.array(testmat['tr'][0][0][1]).squeeze()

y_out=model.predict(X_test, batch_size=512,verbose=1)
plt.plot(y_test,y_out,'ro')
plt.ylabel('model prediction')
plt.xlabel('FACS-seq data')

model.summary()

Using matplotlib backend: MacOSX
6816/6816 [==============================] - 15s    
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 4, 8)          231432      convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 4, 8)          0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 4, 8)          0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 4, 8)          4104        dropout_1[0][0]                  
_____

In [8]:
import scipy
out=y_out.squeeze()
print out.shape
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(y_test, out)
print r_value
print slope
print intercept

(6816,)
0.34013971057
0.12461756668
0.0737930304534


In [ ]:
print history.history.keys()

In [46]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [9]:
model.evaluate(X_test,y_test,show_accuracy=True)


  64/6816 [..............................] - ETA: 16s

/Users/jx/anaconda2/lib/python2.7/site-packages/Keras-1.1.1-py2.7.egg/keras/models.py:666: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


6816/6816 [==============================] - 16s    


[0.24888177172328946, 0.914612676056338]